In [1]:
import SpykeTorch.utils as utils

kernels = [	utils.GaborKernel(window_size = 9, orientation = 45+22.5),
			utils.GaborKernel(9, 90+22.5),
			utils.GaborKernel(9, 135+22.5),
			utils.GaborKernel(9, 180+22.5)]
filter = utils.Filter(kernels, use_abs = True)

In [2]:
def time_dim(input):
	return input.unsqueeze(0)

In [3]:
import SpykeTorch.utils as utils
import SpykeTorch.functional as sf
import torchvision.transforms as transforms

transform = transforms.Compose(
	[transforms.Grayscale(),
	transforms.ToTensor(),
	time_dim,
	filter,
	sf.pointwise_inhibition,
	utils.Intensity2Latency(number_of_spike_bins = 15, to_spike = True)])

In [4]:
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import random

dataset = ImageFolder("/Users/taraapple/Downloads/129")
sample_idx = random.randint(0, len(dataset) - 1)

# plotting the sample image
ax = plt.subplot(1,1,1)
plt.setp(ax, xticklabels=[])
plt.setp(ax, yticklabels=[])
plt.xticks([])
plt.yticks([])
plt.imshow(dataset[sample_idx][0])
plt.show()

In [5]:
import numpy as np

dataset = ImageFolder("/Users/taraapple/Downloads/129", transform) # adding transform to the dataset
plt.style.use('seaborn-white')
plt_idx = 0
sw = dataset[sample_idx][0]
for f in range(4):
	for t in range(5):
		plt_idx += 1
		ax = plt.subplot(5, 5, plt_idx)
		plt.setp(ax, xticklabels=[])
		plt.setp(ax, yticklabels=[])
		if t == 0:
			ax.set_ylabel('Feature ' + str(f))
		plt.imshow(sw[t,f].numpy(),cmap='gray')
		if f == 3:
			ax = plt.subplot(5, 5, plt_idx + 5)
			plt.setp(ax, xticklabels=[])
			plt.setp(ax, yticklabels=[])
			if t == 0:
				ax.set_ylabel('Sum')
			ax.set_xlabel('t = ' + str(t))
			plt.imshow(sw[t].sum(dim=0).numpy(),cmap='gray')
plt.show()

In [6]:
# splitting training and testing sets
indices = list(range(len(dataset)))
random.shuffle(indices)
split_point = int(0.8*len(indices))
train_indices = indices[:split_point]
test_indices = indices[split_point:]
print("Size of the training set:", len(train_indices))
print("Size of the  testing set:", len(test_indices))

Size of the training set: 483
Size of the  testing set: 121


In [7]:
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler

dataset = utils.CacheDataset(dataset)
train_loader = DataLoader(dataset, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(dataset, sampler=SubsetRandomSampler(test_indices))

In [8]:
import SpykeTorch.snn as snn
pool = snn.Pooling(kernel_size = 9, stride = 2)

In [9]:
conv = snn.Convolution(in_channels=4, out_channels=20, kernel_size=30)

In [10]:
stdp = snn.STDP(conv_layer = conv, learning_rate = (0.05, -0.015))

In [11]:
conv.reset_weight()
anti_stdp = snn.STDP(conv_layer = conv, learning_rate = (-0.05, 0.0005))

In [12]:
feature2class = [0] * 10 + [1] * 10
print(feature2class)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [13]:
%%time
print("Starting Reinforcement Learning ...")
for iter in range(150):
	print('\rIteration:', iter, end="")
	for data,targets in train_loader:
		for x,t in zip(data, targets):
			x = pool(x)
			p = conv(x)
			o, p = sf.fire(p, 20, return_thresholded_potentials=True)
			winners = sf.get_k_winners(p, kwta=1, inhibition_radius=0, spikes=o)
			if len(winners) != 0:
				if feature2class[winners[0][0]] == t:
					stdp(x, p, o, winners)
				else:
					anti_stdp(x, p, o, winners)
print()
print("Reinforcement Learning is Done.")

Starting Reinforcement Learning ...
Iteration: 149
Reinforcement Learning is Done.
CPU times: user 2d 10h 50min 3s, sys: 45min 55s, total: 2d 11h 35min 58s
Wall time: 2d 9h 29min 30s


In [14]:
import SpykeTorch.visualization as vis
import torch
feature = torch.stack([k() for k in kernels])
cstride = (1,1)
# C1 Features #
feature,cstride = vis.get_deep_feature(feature, cstride, (9, 9), (2, 2))
# S2 Features #
feature,cstride = vis.get_deep_feature(feature, cstride, (30,30), (1,1), conv.weight)

feature_idx = 0
for r in range(4):
    for c in range(5):
        ax = plt.subplot(4, 5, feature_idx+1)
        plt.xticks([])
        plt.yticks([])
        plt.setp(ax, xticklabels=[])
        plt.setp(ax, yticklabels=[])
        plt.imshow(feature[feature_idx].numpy(),cmap='gray')
        feature_idx += 1
plt.show()

In [15]:
error = 0
silent = 0
total = 0
for data,targets in train_loader:
	for x,t in zip(data, targets):
		total += 1
		x = pool(x)
		p = conv(x)
		o, p = sf.fire(p, 20, return_thresholded_potentials=True)
		winners = sf.get_k_winners(p, kwta=1, inhibition_radius=0, spikes=o)
		if len(winners) != 0:
			if feature2class[winners[0][0]] != t:
				error += 1
		else:
			silent += 1
print("   Train Error:", error/total)
print("Silent Samples:", silent/total)

   Train Error: 0.11801242236024845
Silent Samples: 0.0


In [16]:
error = 0
silent = 0
total = 0
for data,targets in test_loader:
	for x,t in zip(data, targets):
		total += 1
		x = pool(x)
		p = conv(x)
		o, p = sf.fire(p, 20, return_thresholded_potentials=True)
		winners = sf.get_k_winners(p, kwta=1, inhibition_radius=0, spikes=o)
		if len(winners) != 0:
			if feature2class[winners[0][0]] != t:
				error += 1
		else:
			silent += 1
print("    Test Error:", error/total)
print("Silent Samples:", silent/total)

    Test Error: 0.2727272727272727
Silent Samples: 0.0
